## Normal Chain

In [2]:
from json import load
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
parser = StrOutputParser()

template = PromptTemplate.from_template("You are a helpful assistant. Answer the question: {question}")

chain = template | llm | parser

result = chain.invoke({"question": "What is the capital of France?"})
print(result)

The capital of France is Paris.


In [6]:
chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


## Sequential Chain

In [3]:
from json import load
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
parser = StrOutputParser()

seq_template1 = PromptTemplate.from_template("""
Give me a short summary on topic : {topic}""")

seq_template2 = PromptTemplate.from_template("""
Based on the summary, give me 5 key pints on it \n {summary}""")

seq_chain = seq_template1 | llm | parser | seq_template2 | llm | parser

seq_result = seq_chain.invoke({"topic": "Generative AI"})

print(seq_result)



Here are five key points based on the summary of generative AI:

1. **Definition and Focus**: Generative AI is a subset of artificial intelligence dedicated to creating new content (text, images, music, etc.) by learning patterns from existing data.

2. **Techniques Used**: It employs advanced techniques such as deep learning, neural networks, generative adversarial networks (GANs), and transformer models to generate outputs that resemble human creativity.

3. **Diverse Applications**: Generative AI has a wide range of applications, including content creation, art generation, game design, and drug discovery, showcasing its versatility across different fields.

4. **Innovative Possibilities**: The technology offers innovative opportunities for creativity and efficiency in various industries, pushing the boundaries of what machines can create.

5. **Ethical Concerns**: The rise of generative AI raises significant ethical issues, including concerns about copyright infringement, the spread

In [5]:
seq_chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *       

## Parallel Chain

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
parser = StrOutputParser()

par_template1 = PromptTemplate.from_template("""
Give me a short summary on topic : {topic}""")

par_template2 = PromptTemplate.from_template("""
Based on the summary, give me 5 key pints on it \n {topic}""")

par_chain1 = par_template1 | llm | parser
par_chain2 = par_template2 | llm | parser


parallel_chain = RunnableParallel(

  {
    "summary": par_chain1,
    "key_points": par_chain2
  }
)

mer_template = PromptTemplate.from_template("""
  Combine the following information into a comprehensive output:
                                             
  Summary: {summary}
  Key Points: {key_points}

""")

mer_chain = mer_template | llm | parser

final_chain = parallel_chain | mer_chain

par_result = final_chain.invoke({"topic": "Generative AI"})
print(par_result)




**Comprehensive Overview of Generative AI**

Generative AI represents a transformative class of artificial intelligence models designed to create new content—ranging from text and images to music and beyond—by learning patterns from existing data. Utilizing advanced techniques such as deep learning and neural networks, these models, including well-known frameworks like Generative Adversarial Networks (GANs) and transformer-based architectures (e.g., GPT), are capable of producing high-quality outputs that closely mimic human creativity.

### Key Points:

1. **Definition and Functionality**: Generative AI encompasses algorithms that generate new content by analyzing and learning from existing datasets. This technology leverages deep learning methodologies to create outputs that reflect human-like creativity, making it a powerful tool for various creative processes.

2. **Applications**: The applications of Generative AI are vast and diverse, impacting numerous fields. In content creatio

In [9]:
final_chain.get_graph().print_ascii()

        +-----------------------------------+        
        | Parallel<summary,key_points>Input |        
        +-----------------------------------+        
                 **               **                 
              ***                   ***              
            **                         **            
+----------------+                +----------------+ 
| PromptTemplate |                | PromptTemplate | 
+----------------+                +----------------+ 
          *                               *          
          *                               *          
          *                               *          
  +------------+                    +------------+   
  | ChatOpenAI |                    | ChatOpenAI |   
  +------------+                    +------------+   
          *                               *          
          *                               *          
          *                               *          
+-----------------+         

## Conditional Chain

In [15]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain_core.runnables import RunnableParallel, RunnableBranch
from pydantic import BaseModel, Field
from typing import Literal
from dotenv import load_dotenv

load_dotenv()

class ReviewSentiment(BaseModel):
  sentiment: Literal["positve", "negative", "neutral"] = Field(description=" The sentiment of the movie review")

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
str_parser = StrOutputParser()
pydantic_parser = PydanticOutputParser(pydantic_object=ReviewSentiment)

con_template1 = PromptTemplate.from_template("""
    Give me sentiment of the movie review {review} \n
    format_instruction: {format_instruction}
""",
partial_variables={"format_instruction": pydantic_parser.get_format_instructions()}
)

review_chain = con_template1 | llm | pydantic_parser

# print(review_chain.invoke({"review": "This movie was fantastic! The acting was superb and the plot was engaging."}))


pos_template = PromptTemplate.from_template("""
  The review is postive so write a short appreciation: \n {review}
""")

neg_template = PromptTemplate.from_template("""
  The review is negative so write a short critique: \n {review}
""")

neu_template = PromptTemplate.from_template("""
  The review is neutra so write a short balanced remarks: \n {review}
""")

defult_template = PromptTemplate.from_template("""
  I counln not detect the clear sentimen. Provide a netral response: \n {review}
""")


branch_chain = RunnableBranch(
  (lambda x: x.sentiment == "positive", pos_template | llm | str_parser),
  (lambda x: x.sentiment == "negative", neg_template | llm | str_parser),
  (lambda x: x.sentiment == "neutral", neu_template | llm | str_parser),
  defult_template | llm | str_parser
  )

final_con_chain = review_chain | branch_chain

review = "t was okay—some good moments, but nothing particularly memorable. Just an average watch."

print(review_chain.invoke({"review": review}))

final_con_result = final_con_chain.invoke({"review": review})

print(final_con_result)


sentiment='neutral'
The review presents a balanced perspective, highlighting both positive and negative aspects without leaning towards either side. It offers a fair assessment, making it a useful read for those seeking an objective viewpoint.
